The following tools will be used throughout the project.

Catboost
Streamlit
FastAPI
Docker
Table of Contents
· 1. Aim of the Project and Data Set
∘ Data Set
· 2. Data Preprocessing and Model Development
· 3. Interface (Streamlit)
· 4. API (FastAPI)
· 5. Automation (Docker)

https://miro.medium.com/v2/resize:fit:720/format:webp/1*2se448p2nO8Sqcwq_KzrTA.png

You can access the customer data to be used in the project from Kaggle. This dataset consists of 7043 customers with 21 columns (features). It contains customer account information, demographic information, and registered services. The target variable (Churn) provides information on whether the customer has churned.
The dataset is available at the following link: https://community.ibm.com/community/user/businessanalytics/blogs/steven-macko/2019/07/11/telco-customer-churn-1113

# Library

In [ ]:
# @title Install Modules
!pip install numpy pandas matplotlib tensorflow pytorch scikit-learn
!pip install catboost

In [ ]:
# @title Import Modules and Libraries 
import numpy as np
import pandas as pd
import os

from sklearn import metrics
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
from sklearn.metrics import (
    accuracy_score, classification_report, recall_score, confusion_matrix,
    roc_auc_score, precision_score, f1_score, roc_curve, auc
)
from sklearn.preprocessing import OrdinalEncoder

from catboost import CatBoostClassifier, Pool

# Data Set

There are many categorical variables in the dataset. By encoding them numerically, we can use different models with various preprocessing techniques. In this study, we will focus on using the proven Catboost algorithm for handling categorical variables. To enhance model development performance, consider using tools like Optuna for hyperparameter optimization.

In [ ]:
# @title Preprocessing 
